In [1]:
import numpy as np
import arcpy
from arcpy import sa
from arcpy.sa import *

In [4]:
## Построение карт квантилей 5% и 95% 
arcpy.CheckOutExtension("Spatial")

npp_layer = arcpy.Raster("E:/modis_2001_2024/npp_steppe/npp_ecoregions/npp2001_steppe_reclass_eco.tif")
mask_raster = arcpy.Raster("E:/ecoregions_palearctic_steppe/ecoregions.tif")
npp_layer = sa.ExtractByMask(npp_layer, mask_raster)
npp_array = arcpy.RasterToNumPyArray(npp_layer)

# Исключаем NoData
no_data_value = npp_layer.noDataValue
print(f"Значение NoData: {no_data_value}")

if no_data_value is not None:
    mask = (npp_array != 0) & (npp_array != no_data_value)
else:
    mask = (npp_array != 0)
   
npp_array_masked = np.where(mask, npp_array, np.nan)
print(f"Число элементов после применения маски: {np.count_nonzero(~np.isnan(npp_array_masked))}")

if np.isnan(np.nanmin(npp_array_masked)):
    raise ValueError("Массив после маски состоит только из NaN значений.")

min_value = np.nanmin(npp_array_masked)
max_value = np.nanmax(npp_array_masked)
print(f"Минимальное значение: {min_value}, Максимальное значение: {max_value}")
q5 = np.nanpercentile(npp_array_masked, 5)
q95 = np.nanpercentile(npp_array_masked, 95)
print(f"5-й квантиль: {q5}, 95-й квантиль: {q95}")

if q5 >= q95:
    raise ValueError(f"5-й квантиль больше или равен 95-му: {q5} >= {q95}")

remap = RemapRange([
    [float(min_value), float(q5 + 1e-6), 1], # Включаем ровно 5% в 1-й класс
    [float(q5 + 1e-6), float(q95), 2],       # Второй класс (между 5-м и 95-м квантилями)
    [float(q95), float(max_value), 3]        # Включаем ровно 95% в 3-й класс
])

quantile_npp2001 = Reclassify(npp_layer, "VALUE", remap, "NODATA")
final_raster_path = "E:/modis_2001_2024/npp_steppe/npp_ecoregions/quantile_npp2001.tif"
quantile_npp2001.save(final_raster_path)

arcpy.CheckInExtension("Spatial")
arcpy.Delete_management(npp_layer)

Значение NoData: None
Число элементов после применения маски: 79231609
Минимальное значение: 1.0, Максимальное значение: 19256.0
5-й квантиль: 531.0, 95-й квантиль: 5288.0


'CheckedOut'